## 라마3 한국어 뉴스 파인튜닝
금융 뉴스로 부터 종목에 영향을 주는 뉴스인지 판별하는 모델을 파인튜닝하는 과정을 진행해보겠습니다. 실습은 Runpod클라우드에서 A100 SXM GPU를 사용하여 진행하였다고 가정합니다.

###1. 패키지 설치 및 데이터로드
**1) 패키지 설치 및 임포트**

In [ ]:
!pip install torch==2.4.0 transformers==4.45.1 datasets===3.0.1 accelerate==0.34.2 trl==0.11.1 peft==0.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

이제 실습에 사용할 각종 라이브러리의 도구들을 임포트 합니다.

In [ ]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

각 도구들의 쓰임새는 다음과 같습니다.
• load_dataset, Dataset: 다양한 데이터셋을 쉽게 로드하고 처리할 수 있습니다. load_dataset 은 허깅페이스에 업로드 된 데이터셋을 불러오거나 로컬 파일을 로드하는 데 사용되며, Dataset 은 개별 데이터셋 객체를 다룰 때 사용됩니다.

- AutoModelForCausalLM: 허깅페이스 Transformers 라이브러리에서 제공하는 모델 다운로드를 위한 도구입니다. 언어 모델을 로드하는 데 사용됩니다.
- AutoTokenizer: 특정 모델에 맞는 토크나이저를 자동으로 불러오는 도구입니다. 토크나이저는 텍
스트를 언어 모델이 처리할 수 있는 정수 시퀀스로 변환하거나 정수 시퀀스를 다시 텍스트 문자열로 복원하는 역할을 합니다.
- LoraConfig: 이번 실습에서 사용할 학습 방법인 LoRA(Low‑Rank Adaptation) 튜닝을 사용할 때 필
요한 설정값을 정의합니다. 학습 시에 거대 언어 모델 전체를 업데이트하는 것이 아니라, 거대 언어
모델의 특정 부분만 업데이트하여 보다 효율적으로 학습하는 방식입니다. 여기서는 로라 튜닝을 할 때 로라 튜닝과 연관된 각종 설정값들을 결정합니다.
- SFTConfig: 모델을 학습할 때 필요한 다양한 설정값을 정의하는 도구입니다. 학습 과정에서 모델이 어떻게 업데이트될지를 조정하며 여기에는 학습률, 배치 크기, 옵티마이저 등의 설정을 포함합니다. 모델 전체를 학습할 때도 쓰일 수 있지만, 현재 실습과 같이 로라 튜닝에서처럼 모델의 특정부분만 학습하는 방식에서도 사용합니다. SFTConfig 에서 설정하는 이 설정값들은 모델의 학습 성능과 안정성에 큰 영향을 줍니다.
- SFTTrainer: 실제 학습을 수행하는 클래스입니다. 주어진 데이터셋을 이용해 파인 튜닝 과정을 자동으로 수행하며, 특정 부분만 업데이트하는 LoRA 같은 학습 기법도 적용 가능합니다. 모델, 데이터셋, 학습 설정을 한 번에 입력하여 효율적인 학습을 진행할 수 있도록 돕습니다.

**2) 데이터 전처리**
인터넷을 통해 사용할 데이터를 다운로드 하고 특정 형식으로 전처리를 진행해 보겠습니다.

In [ ]:
#1. 허깅 페이스 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/finance_news_summarizer", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/781 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/991 [00:00<?, ? examples/s]

허깅페이스 허브에서 앞서 설명한 금융 뉴스 데이터셋을 불러옵니다. load_dataset() 함수를 사용하는데, "iamjoonfinance_news_summarizer"이라는 데이터셋을 불러옵니다. 이 데이터셋은
우리가 사용할 전체 데이터를 담고 있습니다. 이제 데이터셋의 총 샘플 수를 계산하여 출력합니다.

In [ ]:
# 2. 데이터 전체 크기만 출력
print("전체 데이터 크기: ", len(dataset))

전체 데이터 크기:  991


전체 데이터의 개수는 991 개입니다. 이제 전체 데이터를 학습 데이터와 테스트 데이터로 분할해야 합니다. test_ratio 변수에 0.5 를 할당하여 전체 데이터의 50% 를 테스트 데이터로, 나머지를 학습 데이터로 사용하도록 비율을 지정합니다.

In [ ]:
# 3. train/test 분할 비율설정 (0.5)
test_ratio = 0.5
train_data = []
test_data = []

일반적으로는 머신 러닝에서는 학습 데이터의 양이 더 많고 성능을 평가하기 위한 테스트 데이터의 양을 더 적게 할당합니다. 하지만 현 실습에서는 현재 5:5 분할을 선택하였는데 이는 유료 클라우드인 런팟을 사용하므로 실습 시 과도한 학습 비용을 방지하기 위해서 학습 데이터를 적게 설정하였습니다. 비용 문제에 부담이 없다면 더 많은 학습 데이터를 사용하시기 바랍니다. 그리고 분할된 데이터의 인덱스를 저장할 train_data와 test_data라는 빈 리스트를 생성합니다.


In [ ]:
# 4. 전체 데이터의 인덱스를 train/test로 분할
data_indices = list(range(len(dataset)))
test_size = int(len(data_indices)* test_ratio)

test_data = data_indices[:test_size]
train_data = data_indices[test_size:]

data_indices = list(range(len(dataset))) 코드로 0 부터 데이터셋 길이‑1 까지의 인덱
스 리스트를 생성합니다. test_size = int(len(data_indices)* test_ratio)로 테스트
데이터셋의 샘플 수를 계산합니다. test_data = data_indices[:test_size]로 인덱스 리스
트의 앞부분을 테스트 데이터로 할당합니다. train_data = data_indices[test_size:]로
인덱스 리스트의 뒷부분을 학습 데이터로 할당합니다. 이제 OpenAI 형식으로 데이터를 변환하는
format_data() 함수를 정의합니다.


In [ ]:
# 5. OpenAI format으로 데이터 변환을 위한 함수
def format_data(sample):
  return {
    "messages": [
      {"role": "system", "content": sample["system_prompt"],},
      {"role": "user", "content": sample["user_prompt"],},
      {"role": "assistant", "content": str(sample["assistant"])},
    ],
  }

penAI 형식이란 ChatGPT 를 개발한 OpenAI 에서 만든 데이터의 표준 형식을 말하는데, messages라
는 리스트 안에 각각의 대화를 역할 (role) 과 내용 (content) 으로 구분하여 담는 구조입니다. 이는 앞에서 GPT‑4 API 실습에서 보았던 형식을 의미합니다. system은 AI 의 행동 지침을, user는 사용자의 질문을, assistant는 AI 의 답변을 나타냅니다. 이제 해당 함수를 실제로 적용해봅시다. 앞서 분할한 train_data와 test_data의 각 샘플에 format_data() 함수를 적용하여 최종 데이터셋을 생성합니다.


In [ ]:
# 6. 분할된 데이터를 OpenAI format으로 변환
train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

각 인덱스에 해당하는 데이터를 format_data() 함수를 이용해 OpenAI형식으로 변환하여
train_dataset과 test_dataset에 저장합니다. 이제 최종적으로 만들어진 train_dataset과
test_dataset의 크기를 출력합니다

In [ ]:
# 7. 최종 데이터셋 크기 출력
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")


전체 데이터 분할 결과: Train 496개, Test 495개


앞서 학습 데이터와 테스트 데이터를 5:5 분할하여 개수가 거의 유사하게 학습 데이터가 496 개, 테스트 데이터가 495 개입니다. 이제 OpenAI 형식으로 전처리가 완료된 데이터를 하나 임의로 출력하여 형식을 이해해보겠습니다. 여기서는 학습 데이터 중 임의로 345 번 샘플을 출력해보겠습니다. messages라는 리스트 안에 데이터가 저장되어져 있으므로 이를 출력합니다.

In [ ]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.\n두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.\n큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.\n아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.\n해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 \'없음\' 등을 적어서는 안 됩니다.\n\n만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": False,\n"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}\n\n만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.\n\n답변:\n{"is_stock_related": True,\n"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],\n"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",\n"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주는데 근거가 

role이 system인 경우, content에는 위에서 작성한 시스템 프롬프트가 저장되어져 있습니다.
role이user인 경우, content에는 현 샘플에서의 사용자의 입력이 저장되어져 있습니다. role이
assistant인 경우, content에는 거대 언어 모델이 답변해야 할 내용이 작성되어져 있습니다. 위와
같이 system, user, assistant와 같은 role 과 각각 해당하는 content로 구성된 형태를 OpenAI 형
식이라고 합니다. 이 형식은 학습을 하기 위한 최종 형식이 아니며 전처리를 위한 중간 단계입니다. 다만,
전처리를 위한 중간 단계의 형태로 OpenAI 에서 제안한 형식을 주로 사용하는 것입니다. 학습을 위한 형
식은 뒤의 내용에서 거대 언어 모델의 토크나이저를 통해 한 번 더 전처리를 진행하여 챗 템플릿이 적용되
어야 합니다.
현재 train_dataset과 test_dataset은 데이터 타입이 리스트입니다. 학습을 위해서 타입을
Dataset으로 변경해야 합니다.

In [ ]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


###2. 모델 로드 및 템플릿 적용
이제 학습을 진행할 모델과 토크나이저를 로드합니다. 거대 언어 모델을 학습 시에는 반드시 로드해야 할
것이 크게 두 가지가 있습니다. 바로 학습할 모델과 이 모델에 입력할 데이터를 전처리하기 위한 도구인
토크나이저입니다.

In [ ]:
# 허깅 페이스 모델 ID
model_id = "NCSOFT/Llama-varco-8B-Instruct"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

여기서는 “NCSOFT/Llama‑VARCO‑8B‑Instruct” 라는 모델을 사용할 것입니다. 따라서 모델과 토크나이
저 모두 해당 모델의 이름을 인자로 사용한 것을 알 수 있습니다. 해당 모델은 NCSOFT 에서 공개한 한글
성능이 뛰어난 모델입니다.

토크나이저를 로드하였다면 이제 LLaMA 를 위한 챗 템플릿 (Chat Template) 을 적용해야 합니다. 챗 템
플릿이란 거대 언어 모델이 학습할 때 사용하는 특정한 대화 형식입니다. 우리가 사용하는 거대 언어 모
델들은 이미 학습된 모델이고, 우리는 이를 로드하여 우리의 데이터로 추가적으로 파인 튜닝하고는 합니
다. 이러한 거대 언어 모델은 만들어졌을 당시에 특정 형식에 맞추어서 학습이 된 상태이므로 파인 튜닝
시에도 이 형식을 지켜주어야 합니다

LLaMA 3 의 챗 템플릿은 다음과 같습니다.

In [ ]:
"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
시 스 템 프 롬 프 트 <|eot_id|><|start_header_id|>user<|end_header_id|>
유 저 프 롬 프 트 <|eot_id|><|start_header_id|>assistant<|end_header_id|>
거 대 언 어 모 델 이 해 야 하 는 답 변 <|eot_id|>"""

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n시 스 템 프 롬 프 트 <|eot_id|><|start_header_id|>user<|end_header_id|>\n유 저 프 롬 프 트 <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n거 대 언 어 모 델 이 해 야 하 는 답 변 <|eot_id|>'

위 템플릿에서 주목할 점은 시스템 프롬프트는 <|start_header_id|>system<|
end_header_id|>와 <|eot_id|> 사이에 작성되고, 유저 프롬프트는 <|start_header_id|>
user<|end_header_id|>와 <|eot_id|> 사이에 작성되어져 있고, 어시스턴트의 응답 (실제 LLM
의 응답이 되어야 하는 부분) 은 <|start_header_id|>assistant<|end_header_id|>와
<|eot_id|> 사이에 작성이 되어져 있다는 점입니다. LLaMA 챗 템플릿으로 데이터를 가공하는 방법은
토크나이저의 apply_chat_template()에 OpenAI 형식으로 가공된 데이터를 넣으면 됩니다. 다음
은 학습 데이터 중 0 번 샘플을 챗 템플릿으로 가공한 후 출력하는 코드를 보여줍니다.

In [ ]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.
두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.
해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 '없음' 등을 적어서는 안 됩니다.

만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": False,
"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}

만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": True,
"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],
"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",
"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 존재했다면 여기에 긍정적인 영향을 주

### 3. 로라 학습을 위한 설정값
이제 학습을 위해 각종 설정값들을 지정해야 합니다. 이번 실습에서는 LoRA(Low‑Rank Adaptation) 학습
방식을 사용하여 파인 튜닝을 해볼 것입니다. LoRA 는 거대한 언어 모델을 학습할 때 풀 파인튜닝 (Full
Finetuning) 보다 연산량과 GPU 메모리 사용량을 줄여 학습할 수 있도록 도와주는 방법입니다.

In [ ]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=8,
    target_modules=["q_proj", "v_proj"],
    task_type = "CAUSAL_LM",
)

각 설정값에 대한 설명은 다음과 같습니다.
- lora_alpha: LoRA(Low‑Rank Adaptation) 에서 사용하는 스케일링 계수를 설정합니다. LoRA 의
가중치 업데이트가 모델에 미치는 영향을 조정하는 역할을 하며, 일반적으로 학습 안정성과 관련
이 있습니다.

- lora_dropout: LoRA 적용 시 드롭아웃 확률을 설정합니다. 드롭아웃은 과적합 (overfitting) 을 방
지하기 위해 일부 뉴런을 랜덤하게 비활성화하는 정규화 기법입니다. 0.1 로 설정하면 학습 중 10%
의 뉴런이 비활성화됩니다.

- r: LoRA 의 랭크 (rank) 를 설정합니다. 이는 LoRA 가 학습할 저차원 공간의 크기를 결정합니다. 작
은 값일수록 계산 및 메모리 효율이 높아지지만 모델의 학습 능력이 제한될 수 있습니다

- bias: LoRA 적용 시 편향 (bias) 처리 방식을 지정합니다. “none” 으로 설정하면 편향이 LoRA 에 의
해 조정되지 않습니다. “all” 또는 “lora_only” 와 같은 값으로 변경하여 편향을 조정할 수도 있습니
다.

- target_modules: LoRA 를 적용할 특정 모듈 (레이어) 의 이름을 리스트로 지정합니다. 예제에서
는 “q_proj” 와 “v_proj” 를 지정하여, 주로 Self‑Attention 메커니즘의 쿼리와 값 프로젝션 부분에
LoRA 를 적용합니다.

- task_type: LoRA 가 적용되는 작업 유형을 지정합니다. “CAUSAL_LM” 은 Causal Language
Modeling, 즉 시퀀스 생성 작업에 해당합니다.
다음으로 SFTConfig 를 설정합니다

###4. 학습을 위한 설정 값

In [ ]:
args = SFTConfig(
  output_dir="llama3-8b-summarizer-ko",     # 저장될 디렉토리와 저장소 ID
  num_train_epochs=3,                       # 학습할 총 에포크 수
  per_device_train_batch_size=2,            # GPU당 배치 크기
  gradient_accumulation_steps=2,            # 그래디언트 누적 스텝 수
  gradient_checkpointing=True,              # 메모리 절약을 위한 체크 포인팅
  optim="adamw_torch_fused",                # 최적화기
  logging_steps=10,                         # 로그 기록 주기
  save_strategy="steps",                    # 저장 전략
  save_steps=50,                            # 저장 주기
  bf16=True,                                # bfloat16 사용
  learning_rate=1e-4,                       # 학습률
  max_grad_norm=0.3,                        # 그래디언트 클리핑
  warmup_ratio=0.03,                        # 워밍업 비율
  lr_scheduler_type="constant",             # 고 정 학 습 률
  push_to_hub=False,                        # 허브 업로드 안함
  remove_unused_columns=False,
  dataset_kwargs={"skip_prepare_dataset": True},
  report_to=None
)


각 설정값에 대한 상세 설명은 다음과 같습니다.
- output_dir: 학습 결과가 저장될 디렉토리 또는 모델 저장소의 이름을 지정합니다. 이 디렉토리에
학습된 모델 가중치, 설정 파일, 로그 파일 등이 저장됩니다.

- num_train_epochs: 모델을 학습시키는 총 에포크 (epoch) 수를 지정합니다. 에포크는 학습 데이
터 전체를 한 번 순회한 주기를 의미합니다.

- per_device_train_batch_size: GPU 한 대당 사용되는 배치 (batch) 의 크기를 설정합니다. 배치크기는 모델이 한 번에 처리하는 데이터 샘플의 수를 의미합니다.

- gradient_accumulation_steps: 그래디언트를 누적할 스텝 (step) 수를 지정합니다. 이 값이 2 로
설정된 경우, 두 스텝마다 그래디언트를 업데이트합니다.

- gradient_checkpointing: 그래디언트 체크포인팅을 활성화하여 메모리를 절약합니다.

- optim: 학습 시 사용할 최적화 알고리즘을 설정합니다. adamw_torch_fused 는 PyTorch 의 효율
적인 AdamW 최적화기를 사용합니다.

- logging_steps: 로그를 기록하는 주기를 스텝 단위로 지정합니다.

- save_strategy 와 save_steps: 모델을 저장하는 전략과 주기를 설정합니다.

- bf16: bfloat16 정밀도를 사용하도록 설정합니다.

- learning_rate: 학습률을 지정합니다.

- max_grad_norm: 그래디언트 클리핑의 임계값을 설정합니다.

- warmup_ratio: 학습 초기 단계에서 학습률을 선형으로 증가시키는 워밍업 비율을 지정합니다.

- lr_scheduler_type: 학습률 스케줄러의 유형을 설정합니다. “constant” 는 학습률을 일정하게 유
지합니다.


### 5. 정수 인코딩
거대 언어 모델에 학습 데이터를 전달하기 전에 학습 데이터는 수치화 작업을 거칩니다. 더 정확히는 텍
스트 데이터는 전부 거대 언어 모델이 이해할 수 있는 정수 데이터로 변환되어야 합니다. 이 과정을 일반
적으로 인코딩 (encoding) 이라 부르며 앞서 로드한 토크나이저를 통해 가능합니다. 예를 통해 인코딩 과
정을 이해해보겠습니다. 거대 언어 모델을 학습할 때에는 입력과 출력 두 가지가 존재해야 합니다. 여기
서는 입력과 출력의 각 변수명을 input_ids와 labels라고 해봅시다. 또한, 모델이 학습해야 하는 데
이터는 다음과 같다고 가정하겠습니다.

In [ ]:
"""-시스템 프롬프트: "당신은 친절한 AI 어시스턴트입니다."
- 유저 프롬프트: "안녕하세요, 오늘 날씨는 어떤가요?"
- 거대 언어 모델의 응답: "안녕하세요! 오늘 날씨는 맑고 화창합니다."""

'-시스템 프롬프트: "당신은 친절한 AI 어시스턴트입니다."\n- 유저 프롬프트: "안녕하세요, 오늘 날씨는 어떤가요?"\n- 거대 언어 모델의 응답: "안녕하세요! 오늘 날씨는 맑고 화창합니다.'

이 데이터를 학습하기 위해서는 앞서 진행된 바와 같이 채팅 템플릿을 적용해야 합니다.

In [ ]:
"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 친절한 AI 어시스턴트입니다..<|eot_id|><|start_header_id|>user<|end_header_id|>

안녕하세요, 오늘 날씨는 어떤가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

안녕하세요! 오늘 날씨는 맑고 화창합니다.<|eot_id|>"""


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n당신은 친절한 AI 어시스턴트입니다..<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n안녕하세요, 오늘 날씨는 어떤가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n안녕하세요! 오늘 날씨는 맑고 화창합니다.<|eot_id|>'

In [ ]:
input_ids = [
    128000, # <|begin_of_text|>
    128006, 9125, 128007, 198, # <|start_header_id|>system<|end_header_id|> (줄 바꿈)
    22173, 13, 126808, 49816, 33302, 23239, 18966, 13, # 당 신 은 친 절 한 AI 어 시 스 턴 트 입 니 다.
    128009, # <|eot_id|>
    128006, 882, 128007, 198, # <|start_header_id|>user<|end_header_id|> (줄 바 꿈)
    118145, 11, 24482, 1174, 107485, 102823, 64337, 30, # 안 녕 하 세 요 , 오 늘 날 씨 는어 떤 가 요?
    128009, # <|eot_id|>
    128006, 78191, 128007, 198, # <|start_header_id|>assistant<|end_header_id|> (줄 바 꿈)
    118145, 0, 24482, 1174, 107485, 102823, 64337, 107823, 108562, 13, # 안 녕 하 세요! 오 늘 날 씨 는 맑 고 화 창 합 니 다.
    128009 # <|eot_id|>
]


In [ ]:
labels = [
    -100, # <|begin_of_text|>
    -100, -100, -100, -100, # <|start_header_id|>system<|end_header_id|> (줄 바 꿈)
    -100, -100, -100, -100, -100, -100, -100, -100, # 당 신 은 친 절 한 AI 어 시 스 턴 트입 니 다.
    -100, # <|eot_id|>
    -100, -100, -100, -100, # <|start_header_id|>user<|end_header_id|> (줄 바 꿈)
    -100, -100, -100, -100, -100, -100, -100, -100, # 안 녕 하 세 요 , 오 늘 날 씨 는 어떤 가 요?
    -100, # <|eot_id|>
    -100, -100, -100, -100, # <|start_header_id|>assistant<|end_header_id|> (줄 바꿈)
    118145, 0, 24482, 1174, 107485, 102823, 64337, 107823, 108562, 13, # 안 녕 하 세요! 오 늘 날 씨 는 맑 고 화 창 합 니 다.
    128009 # <|eot_id|>
]


### 정수 인코딩 적용하기
거대 언어 모델에 학습 데이터를 전달하기 전에 학습 데이터는 수치화 작업을 거칩니다. 텍스트 데이터는
전부 거대 언어 모델이 이해할 수 있는 정수 데이터로 변환되어야 합니다. 이 과정을 일반적으로 인코딩
(encoding) 이라 부르며 앞서 로드한 토크나이저를 통해 가능합니다.

In [ ]:
def collate_fn(batch):
  new_batch = {
      "input_ids": [],
      "attention_mask": [],
      "labels": []
  }

  for example in batch:
    messages = example["messages"]

    # LLaMa 3 채팅 템플릿 적용 (시작 토큰 포함)
    prompt = "<|begin_of_text|>"
    for msg in messages:
      role = msg["role"]
      content = msg["content"].strip()
      prompt += f"<|start_header_id|> {role} <|end_header_id|>\n{content}<|eot_id|>"

    # 마지막 assistant 메시지는 응답으로 간주하고 레이블에 포함
    text = prompt.strip()

    # 토큰화
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=max_seq_length,
        padding=False,
        return_tensors=None,
    )

    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]
    labels = [-100] * len(input_ids)

    # assistant 응답의 시작 위치 찾기
    assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
    assistant_tokens = tokenizer.encode(assistant_header, add_special_tokens=False)
    eot_token = "<|eot_id|>"
    eot_tokens = tokenizer.encode(eot_token, add_special_tokens=False)

    # 레이블 범위 지정
    i = 0
    while i <= len(input_ids) - len(assistant_tokens):
      if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
        start = i + len(assistant_tokens)
        end = start
        while end <= len(input_ids) - len(eot_tokens):
          if input_ids[end:end + len(eot_tokens)] == eot_tokens:
            break
          end += 1
        for j in range(start, end):
          labels[j] = input_ids[j]
        for j in range(end, end + len(eot_tokens)):
          labels[j] = input_ids[j] # <|eot_id|> 토 큰 도 포 함
        break
      i += 1
    new_batch["input_ids"].append(input_ids)
    new_batch["attention_mask"].append(attention_mask)
    new_batch["labels"].append(labels)


  # 패딩 처리
  max_length = max(len(ids) for ids in new_batch["input_ids"])
  for i in range(len(new_batch["input_ids"])):
    pad_len = max_length - len(new_batch["input_ids"][i])
    new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * pad_len)
    new_batch["attention_mask"][i].extend([0] * pad_len)
    new_batch["labels"][i].extend([-100] * pad_len)

  for k in new_batch:
    new_batch[k] = torch.tensor(new_batch[k])

  return new_batch

collate_fn(batch) 함수는 자연어 처리 모델 학습을 위해 데이터를 전처리하는 역할을 수행합니
다. 해당 함수 내에서 input_ids는 전체 대화를 챗 템플릿 이후에 정수 인코딩을 적용한 바꾼 결과물입
니다. 토크나이저는 "<|start_header_id|>"와 "<|end_header_id|>", "<|eot_id|>"
같은 특수 토큰을 포함한 모든 텍스트를 숫자로 변환합니다. 이렇게 변환된 숫자들이 모델에 입력됩니다.
코드에서는 assistant_tokens와 eot_tokens를 먼저 인코딩해서 이 토큰들의 숫자값을 미리 준
비해둡니다.
labels는 모델이 실제로 생성해내야 할 목표값입니다. 이 코드의 핵심은 assistant가 말한 부분만
골라서 학습시키는 것입니다. assistant가 말한 부분은 input_ids의 값을 그대로 labels에 복사
하고, 나머지는 전부 ‑100 으로 채웁니다. ‑100 은 PyTorch 가 학습에서 무시하는 특별한 값입니다.
코 드 는 while 문 을 사 용 해 서 input_ids 안 에 서 "<|start_header_id|>assistant<|
end_header_id|>\n"로 시작하는 부분을 찾습니다. 이 부분부터 "<|eot_id|>"가 나올 때까지
가 assistant의 응답입니다. 이 구간의 토큰들만 labels에 복사하고 나머지는 전부 ‑100 을 넣습니
다. 이렇게 하면 모델은 assistant의 응답만 학습하게 됩니다.
임의의 샘플에 대해서 실제 전처리가 잘 진행되는지 확인해봅시다. 학습 데이터 중 0 번 인덱스를 가진 첫
번째 샘플에 대해서 collate_fn에 통과시켜서 결과를 확인합니다.

In [ ]:
# 데이터의 최대 길이 제한
max_seq_length=8192

# collate_fn 테스트 (배치크기 1. 즉, 데이터 1개에 대히서 전처리 진행해보기)
example = train_dataset[0]
batch = collate_fn([example])
print("\n처리된 배치 데이터:")
print("입력 ID 형태: ",batch["input_ids"].shape)
print("어텐션 마스크 형태: ", batch["attention_mask"].shape)
print("레이블 형태: ", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태:  torch.Size([1, 2887])
어텐션 마스크 형태:  torch.Size([1, 2887])
레이블 형태:  torch.Size([1, 2887])


먼저 데이터의 최대 길이를 8192 로 제한합니다. 이는 이보다 긴 데이터들은 임의로 길이를 자르는 것을
의미합니다. 그 후 학습 데이터의 첫번째 샘플을 collate_fn에 전달하여 함수의 전처리 결과를 얻어
batch라는 변수에 저장합니다. 이 전처리 결과에는 총 3 개의 키가 존재합니다. 바로 input_ids와
labels와 attention_mask입니다. 먼저 첫번째로 input_ids를 출력해보겠습니다.

In [ ]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[128000, 128006, 1887, 220, 128007, 198, 65895, 83628, 34804, 56773, 125441, 111068, 25941, 123103, 99458, 88708, 19954, 126652, 56773, 16969, 111068, 25941, 117469, 106478, 102517, 44005, 104193, 123061, 111068, 25941, 106478, 102517, 21121, 80052, 627, 103097, 109521, 111964, 127491, 122625, 119097, 108859, 101480, 93917, 101868, 56069, 13094, 168, 235, 105, 21028, 11240, 106612, 77437, 43139, 114839, 16582, 119978, 627, 65993, 108, 103386, 36092, 112, 102260, 109055, 19954, 105642, 103386, 36092, 112, 102260, 105880, 103607, 34609, 113348, 45618, 49085, 88525, 96677, 119978, 13, 127063, 11240, 56069, 113890, 18359, 62085, 99742, 102893, 105365, 102467, 112215, 124005, 13, 106725, 27796, 56773, 21028, 16582, 119978, 627, 54059, 54542, 11240, 57575, 106603, 907, 16969, 67890, 30426, 115790, 19954, 95713, 61938, 13, 67890, 56154, 115790, 18359, 106725, 103607, 22035, 100711, 119978, 13, 95713, 67890, 30426, 115790, 19954, 106725, 103607, 104834, 24486, 907, 18918, 104

collate_fn은 들어온 데이터에 대해서 챗 템플릿을 적용 후에 정수 인코딩까지 진행합니다

In [ ]:
print("레이블에 대한 정수 인코딩 결과:")
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

출력 결과를 살펴보면 입력에 대한 정수 인코딩 결과와 레이블에 대한 정수 인코딩 결과는 길이는 동일합
니다. 하지만 레이블에 대한 정수 인코딩 결과에서는 거대 언어 모델의 실제 답변 부분을 제외하고는 전부
‑100 으로 채워진 값이 출력됩니다. 디코딩된 결과도 확인할 수 있습니다. 먼저 input_ids를 디코딩한
결과입니다.


In [ ]:
# 디코딩된 input_ids 출력
decoded_text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False,
)
print("\ninput_ids 디코딩 결과:")
print(decoded_text)



input_ids 디코딩 결과:
<|begin_of_text|><|start_header_id|> system <|end_header_id|>
당신은 주어진 뉴스로부터 종목에 영향을 주는 뉴스인지 판별하는 금융 뉴스 판별기입니다.
두 가지 답변 케이스가 존재하며 무조건 파이썬의 dictionary 형식으로 작성하십시오.
큰 따옴표 사이에 다른 따옴표들을 적으려고 시도하지 마십시오. 이는 dictionary 파싱을 실패하게 하는 원인이 됩니다. 따라서 주의하십시오.
아래 dictionary에서 각 value는 지시사항에 해당합니다. 지사사항을 따라 적지마십시오. 해당 지시사항에 따라 적절한 value를 채워넣으십시오.
해당사항이 없다면 빈 문자열 또는 빈 리스트로 적어야 합니다. 임의로 '없음' 등을 적어서는 안 됩니다.

만약 해당 뉴스가 특정 종목(회사)이 언급되지 않거나, 특정 종목(회사)와 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": False,
"summary": "여기에는 해당 뉴스를 요약해서 요약문을 작성하십시오"}

만약 해당 뉴스가 특정 종목(회사)들과 연관되었거나, 특정 종목(회사)과 아무런 연관이 없는 뉴스일 경우에는 아래와 같이 작성합니다.

답변:
{"is_stock_related": True,
"positive_impact_stocks": ["파이썬 문자열 리스트의 형태로 이 뉴스가 긍정적인 영향을 줄것으로 추정되는 종목들의 이름을 작성하십시오. 약자로 적거나 별명으로 적지마십시오. 종목명으로 추정되는 한글명을 적으십시오. 뉴스로부터 추정할 수 있는 정확한 풀네임으로 적으십시오. 만약, 존재하지 않는다면 빈 리스트로 작성하십시오."],
"reason_for_positive_impact": "위의 종목들이 해당 뉴스로부터 긍정적인 영향을 받을 것으로 추정한 이유를 여기에다가 작성하십시오",
"positive_keywords": ["긍정적인 영향을 줄 것으로 추정되는 종목들이 

챗 템플릿이 적용된 결과를 확인할 수 있습니다. 이제 labels의 디코딩 된 결과를 확인해보겠습니다.
단, ‑100 은 실제 맵핑되는 토큰이 없으므로 ‑100 은 제외하고 출력합니다.

In [ ]:
# -100이 아닌 부분만 골라 디코딩
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\nlabels 디코딩 결과 (-100 제외):")
print(decoded_labels)


labels 디코딩 결과 (-100 제외):



시스템 프롬프트와 유저 프롬프트는 제외하고 어시스턴트의 응답만 출력됩니다. collate_fn을 통과
한 후의 input_ids와 labels에 대해서 배웠습니다. 이제 마지막으로 attention_mask에 대해서
설명해보겠습니다

### 6. 어텐션 마스크 확인
어텐션 마스크에 대해서 이해하기 위해 배치 연산에 대해서 설명해보겠습니다. 배치 연산은 다수의 샘플
을 동시에 처리하는 것을 말하며 배치 크기란 모델이 한 번에 학습하는 데이터 샘플의 수를 의미합니다.
예를 들어 배치 크기가 3 이면, 모델은 세 개의 데이터 샘플을 동시에 처리합니다. 이렇게 병렬적으로 학
습하면 계산 효율성이 높아지고 학습 속도가 빨라지는 이점이 있어서 학습 시에는 주로 배치 학습을 합니
다. 실제로 위에서 SFTConfig() 설정 안에서 per_device_train_batch_size=2의 값을 사용
하였던 것을 상기합니다. 이는 GPU 장비당 배치 크기가 2 라는 의미인데, 이번 실습에서 GPU 장비는 1 개
를 사용합니다. 다시 말해 이번 실습에서의 배치 크기는 2 입니다. 그런데 배치 연산을 위해서는 배치 내의 모든 샘플의 길이가 동일해야 한다는 조건이 붙습니다. 하지만 자연어 처리에서 각각의 샘플은 길이가 다양합니다. 첫번째 샘플은 장문의 문장일 수도 있고, 두번째 샘플은 짧은 문장일 수도 있습니다. 즉, 자연어 처리에서 전체 데이터 내의 샘플들끼리 모두 길이가 똑같을 가능성은 거의 없다는 겁니다.

그런데 배치 연산을 위해서는 배치 내의 모든 샘플의 길이가 동일해야 한다는 조건이 붙습니다. 하지만 자
연어 처리에서 각각의 샘플은 길이가 다양합니다. 첫번째 샘플은 장문의 문장일 수도 있고, 두번째 샘플
은 짧은 문장일 수도 있습니다. 즉, 자연어 처리에서 전체 데이터 내의 샘플들끼리 모두 길이가 똑같을 가
능성은 거의 없다는 겁니다. 예를 들어 배치 크기 3 인 경우를 가정해봅시다.


In [ ]:
"""샘플1: "인공지능이란 무엇인가요?"
→ [101, 4089, 8024, 6356, 102] (5 토 큰)
샘플2: "오늘 날씨가 정말 좋네요"
→ [101, 3157, 2533, 4120, 2642, 8730, 6824, 102] (8 토 큰)
샘플3: "딥러닝 모델을 학습 시키는 방법을 알려주세요."
→ [101, 2982, 3478, 4567, 2053, 8276, 5036, 2355, 4602, 7312, 102] (11 토 큰)
"""

위 3 개의 샘플을 배치 크기 3 으로 데이터를 처리하려면 임의로 데이터의 길이를 조절하는 기술’ 패딩
(Padding)’ 을 사용하여 3 개의 샘플의 길이를 동일하게 맞추어야 합니다. ’ 패딩 (Padding)’ 이란 길이가
짧은 샘플들을 긴 샘플들의 길이에 강제로 맞추는 기술로 보통 뒤에 숫자 0 을 붙여서 해결합니다. 현재는
각 샘플의 길이기 5, 8, 11 이므로 위 예시에서는 가장 긴 샘플에 세번째 샘플로 길이는 11 입니다. 이에 따
라서 다른 샘플들에 숫자 0 을 추가합니다.

In [ ]:
"""샘플1: [101, 4089, 8024, 6356, 102, 0, 0, 0, 0, 0, 0] (5 실제 + 6 패딩)
샘플2: [101, 3157, 2533, 4120, 2642, 8730, 6824, 102, 0, 0, 0] (8 실제 + 3 패딩)
샘플3: [101, 2982, 3478, 4567, 2053, 8276, 5036, 2355, 4602, 7312, 102] (11 실제 토큰)"""

샘플 1 과 샘플 2 에 뒤에 숫자 0 들을 강제로 추가하여 이렇게 하면 모든 샘플이 길이 11 이 되도록 했습니
다. 이제 샘플 3 개가 길이가 모두 동일하므로 배치 크기 3 의 배치 연산이 가능해졌습니다. 학습 과정에서
는 저 3 개가 한 번에 묶여서 input_ids로 사용될 것입니다.
그런데 패딩을 위해 추가된 0 은 실제 의미가 있는 정수는 아니고 순전히 길이를 맞추기 위한 용도로 아
무 의미도 없는 정수입니다. 모델이 학습되는 과정에서 패딩 용도로 들어간 정수 0 에 대해서 어떠한
의미를 부여하고 학습을 할 필요가 없습니다. 따라서 모델에게 여기까지는 실제 의미가 있는 토큰들
이고 여기서부터는 실제 의미가 없는 토큰 (숫자 0 으로 채워진 구간) 들임을 명시적으로 알려줘야 합
니다. 이를 위해 사용하는 것이 어텐션 마스크입니다. 어텐션 마스크는 input_ids, labels와 함께
attention_mask라는 값으로 함께 모델의
입력으로 사용됩니다. 어텐션 마스크는 실제 패딩 토큰이
아니었던 구간은 1, 패딩 토큰인 구간은 0 으로 채워진 값입니다. 예를 들어 위의 예시에 따르면 어텐션 마
스크는 아래의 값을 가집니다.


In [ ]:
"""샘플1 마스크: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
샘플2 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
샘플3 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

"""

실제 출력을 통해 이해해봅시다. 두 개의 데이터를 동시에 collate_fn에 전달하여 배치 크기 2 인 상
황에 대해서 input_ids와 labels와 attention_mask가 어떤 값을 가지는지 확인해볼 것입니다.
먼저 학습 데이터 중 0 번 데이터와 1 번 데이터의 길이를 확인해봅시다. 두 개의 데이터 각각을 챗 템플릿
을 적용하고, 각각에 대해서 총 토큰의 수를 확인하면 될 것입니다.


In [ ]:
# 0번과 1번 데이터의 길이 확인
example0 = train_dataset[0]
example1 = train_dataset[1]

In [ ]:
# 개별 길이 확인 (토큰화 후)
tokenized0 = tokenizer(
    # 전체 처리 과정과 동일하게 전체 대화를 토큰화
    "<|begin_of_text|>" + "".join([f"<|start_header_id|>{msg['role']}<|end_header_id|>\n{msg['content'].strip()}<|eot_id|>"
     for msg in example0["messages"]]),
    truncation=True,
    max_length=max_seq_length,
    padding=False,
    return_tensors=None,
)

tokenized1 = tokenizer(
    # 전체 처리 과정과 동일하게 전체 대화를 토큰화
    "<|begin_of_text|>" + "".join([f"<|start_header_id|>{msg['role']}<|end_header_id|>\n{msg['content'].strip()}<|eot_id|>"
    for msg in example1["messages"]]),
    truncation=True,
    max_length=max_seq_length,
    padding=False,
    return_tensors=None,
)
print(f"0번 데이터 길이: {len(tokenized0['input_ids'])}")
print(f"1번 데이터 길이: {len(tokenized1['input_ids'])}")

0번 데이터 길이: 2884
1번 데이터 길이: 974


챗 템플릿을 적용했을 때 0 번 데이터의 길이는 2,884 이며 1 번 데이터의 길이는 974 입니다. 다시 말해 이
두 개의 데이터 길이는 서로 상이하며 첫번째 데이터의 길이가 훨씬 긴 상황입니다. 이제 이 두 개의 데이
터를 동시에 collate_fn에 전달해보겠습니다. 즉, 두 개의 데이터가 배치 크기 2 로 전달되는 상황입
니다.

In [ ]:
# 데이터의 최대 길이 제한
max_seq_length = 8192

# 배치로 처리하여 어텐션 마스크 비교
batch = collate_fn([example0, example1])

데이터의 최대 길이를 8192 로 정하고 두 개의 데이터를 리스트로 묶어서 collate_fn에 전달하여 그 결
과를 batch라는 변수에 저장했습니다. 현재 batch에는 input_ids, labels, attention_mask
라는 키 값들이 존재합니다. 먼저 input_ids와 attention_mask의 크기 (shape) 를 출력해보겠습
니다.

In [ ]:
print("\n배치 처리 후:")
print(f"입력 ID 형태: {batch['input_ids'].shape}")
print(f"어텐션 마스크 형태: {batch['attention_mask'].shape}")



배치 처리 후:
입력 ID 형태: torch.Size([2, 2887])
어텐션 마스크 형태: torch.Size([2, 2887])


input_ids와 attention_mask 모두 2 행 2,884 열이라는 크기가 출력됩니다. 또는 길이가 2,884 인
데이터가 2 개가 있다고 해석해볼 수도 있을 것입니다. 이 말은 1 번 데이터가 0 번 데이터의 길이로 강제
로 맞춰졌음을 의미하며 내부에서 패딩이 이루어졌음을 의미합니다. 이제 1 번 데이터의 input_ids와
attention_mask에는 길이 974 에서 강제로 길이 2,884 로 맞추기 위해서 그만큼 뒤에 숫자 0 이 1,910
개가 추가되었을 것입니다.
실 제 로 0 번 데 이 터 의 attention_mask에 서 의 숫 자 1 과 0 의 개 수 와 1 번 데 이 터 의
attention_mask에서의 숫자 1 과 0 의 개수를 출력해봅시다.

In [ ]:
print(f"0번 샘 플 어 텐 션 마 스 크 1의 개 수: {batch['attention_mask'][0].sum().item()}")
print(f"0번 샘 플 어 텐 션 마 스 크 0의 개 수: {(batch['attention_mask'][0] == 0).sum().item()}")
print(f"1번 샘 플 어 텐 션 마 스 크 1의 개 수: {batch['attention_mask'][1].sum().item()}")
print(f"1번 샘 플 어 텐 션 마 스 크 0의 개 수: {(batch['attention_mask'][1] == 0).sum().item()}")

0번 샘 플 어 텐 션 마 스 크 1의 개 수: 2887
0번 샘 플 어 텐 션 마 스 크 0의 개 수: 0
1번 샘 플 어 텐 션 마 스 크 1의 개 수: 977
1번 샘 플 어 텐 션 마 스 크 0의 개 수: 1910


0 번 샘플과 1 번 샘플의 어텐션 마스크를 직접 출력한 결과 0 번 샘플의 어텐션 마스크의 1 의 개수는 2,884
개입니다. 길이가 2,884 개이므로 모두 1 으로 채워져 있다는 의미입니다. 그도 그럴게 0 번 샘플은 패딩
이 되지 않았기 때문입니다. 반면, 1 번 샘플의 어텐션 마스크의 1 의 개수는 974 개이고 0 의 개수는 1,910
개인데 이는 앞은 974 개의 1 로 채워져있지만 뒷 부분은 0 이 1,910 개가 채워져있음을 의미합니다. 이처
럼 어텐션 마스크는 input_ids에서 어느 구간까지가 패딩이 아니고, 어느 구간이 패딩인지를 알려주
는 역할을 합니다. 학습 시 모델에는 input_ids, labels, attention_mask가 모두 입력으로 사용
됩니다.


###7. 학습하기
이제 전처리가 끝났으므로 실제 학습을 진행합니다.

In [ ]:
trainer = SFTTrainer(
model=model,
args=args,
max_seq_length=max_seq_length, # 최대 시퀀스 길이 설정
train_dataset=train_dataset,
data_collator=collate_fn,
peft_config=peft_config,
)


학습을 시작하면 3 번의 에포크를 도는 동안 총 372 번의 업데이트가 필요하다고 출력됩니다. 왜 3 번
의 에포크 동안 372 번 업데이트를 하는 것일까요? 앞서 학습 데이터의 개수는 496 개였습니다. 그리
고 SFTConfig() 설정에서 num_train_epochs=3, per_device_train_batch_size=2,
gradient_accumulation_steps=2 값 을 설 정 한 것 을 기 억 합 시 다. num_train_epochs
=3의 값 의 설 정 에 따 라 서 전 체 데 이 터 에 대 한 학 습 횟 수. 즉, 에 포 크 는 총 3 회 입 니 다.
per_device_train_batch_size=2는 GPU 장비 당 배치 크기를 의미합니다. 현재 GPU 는 1 개 사
용하고 있으므로 다시 말해 배치 크기가 2 라는 의미입니다. gradient_accumulation_steps는
배치를 몇 개까지 모아두었다가 한 번에 모델을 업데이트 할 것이냐를 의미합니다. 정리하면 다음과 같습
니다.

- 배치 크기 (batch size): 2 ‑ 한 번에 처리하는 데이터 샘플의 수입니다. 메모리 효율을 위해 전체 데
이터를 작은 배치로 나누어 처리하며, 여기서는 2 개씩 묶어서 처리합니다.
- 누적 단계 (accumulation steps): 2 ‑ 모델을 실제로 업데이트하기 전에 여러 배치의 정보를 모으
는 수입니다. 여기서는 2 개의 배치 (총 4 개의 샘플) 를 처리한 후에야 실제 모델 업데이트가 일어납
니다.
- 에포크 1 회당 업데이트 횟수: 496 ÷ (2 × 2) = 124 회 ‑ 한 에포크에서 모델이 업데이트되는 횟수입
니다. 전체 데이터 496 개를 유효 배치 크기 4(배치 크기 2 × 누적 단계 2) 로 나누면 124 번의 업데이
트가 발생합니다.
- 총 업데이트 계산 방법: (데이터 크기 × 에포크) ÷ (배치 크기 × 누적 단계) ‑ 학습 과정 전체에서 발
생하는 모델 업데이트의 총 횟수를 계산하는 공식입니다. 전체 처리 샘플 수를 유효 배치 크기로 나
눕니다.
- 총 업데이트 계산 과정: (496 × 3) ÷ (2 × 2) = 1,488 ÷ 4 = 372 ‑ 3 개의 에포크 동안 총 1,488 개의 샘
플이 처리되고, 유효 배치 크기인 4 개의 샘플마다 한 번씩 모델이 업데이트되므로 총 372 번의 모
델 업데이트가 발생합니다

###8. 테스트 데이터 준비하기
테스트 데이터를 전처리하여 토크나이저의 apply_chat_template()를 사용하여 챗 템플릿을
적용해봅시다. 챗 템플릿 적용 시에는 시스템 프롬프트와 유저 프롬프트에 이어 입력의 뒤에 <|
start_header_id|>assistant<|end_header_id|>\n가 부착되어서 넣는 것이 좋습니다.
그러면 모델이 조금 더 안정적으로 답변을 생성합니다. 이는 앞서 챗 템플릿에서 설명했던 생성 프롬프트
(generation prompt) 에 해당됩니다.


In [ ]:
prompt_lst = []
label_lst = []

for messages in test_dataset["messages"]:
  text = tokenizer.apply_chat_template(messages, tokenize=False,
    add_generation_prompt=False)
  input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|
    start_header_id|>assistant<|end_header_id|>\n'
  label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
  prompt_lst.append(input)
  label_lst.append(label)



챗 템플릿을 적용하고 테스트 데이터의 입력과 레이블을 분리했습니다. 임의로 200 번 샘플을 출력해보
겠습니다.


In [ ]:
print(prompt_lst[200])

In [ ]:
print(label_lst[200])

테스트 데이터에 대해서 챗 템플릿을 적용하고 생성 프롬프트를 부착하는 전처리가 끝났습니다. 이제 파
인 튜닝 모델의 입력으로 테스트 데이터를 넣어 모델의 예측을 확인해봅시다.

###9. 파인튜닝 모델 테스트
AutoPeftModelForCausalLM()의 입력으로 LoRA Adapter 가 저장된 체크포인트의 주소를 넣으
면 LoRA Adapter 가 기존의 LLM 과 부착되어 로드됩니다. 이 과정은 LoRA Adapter 의 가중치를 사전 학
습된 언어 모델 (LLM) 에 통합하여 미세 조정된 모델을 완성하는 것을 의미합니다.


In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
peft_model_id = "llama3-8b-summarizer-ko/checkpoint-372"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)


peft_model_id 변수는 미세 조정된 가중치가 저장된 체크포인트의 경로를 나타냅니다. "llama3
-8b-summarizer-ko/checkpoint-372"는 LoRA Adapter 가중치가 저장된 위치로, 이 경로에서
해당 가중치를 불러옵니다.
fine_tuned_model은 AutoPeftModelForCausalLM.from_pretrained 메서드를통해 체
크포인트를 로드하여 생성됩니다. 이 메서드는 LLM 과 LoRA Adapter 를 결합하고, 최적화된 설정으로 모
델을 메모리에 로드합니다. device_map="auto" 옵션은 모델을 자동으로 GPU 에 배치합니다.
pipeline은 허깅페이스의 고수준 유틸리티로 모델과 토크나이저만 전달하면 모델로 전달한 입력에
대해서 바로 답변을 반환하는 일종의 LLM 호출 함수를 자동으로 만들어주는 역할을 합니다. 이 코드에
서 사용된 pipeline("text-generation")은 텍스트 생성 작업을 수행하기 위한 파이프라인 객
체를 생성합니다.


In [ ]:
eos_token = tokenizer("<|eot_id|>",add_special_tokens=False)["input_ids"][0]
def test_inference(pipe, prompt):
  outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
  return outputs[0]['generated_text'][len(prompt):].strip()


eos_token은 모델이 텍스트 생성을 멈추는 시점을 알려주는 특수 토큰, 구체적으로는 종료 토큰의 정
수를 저장하는 변수입니다. 종료 토큰이란 모델이 해당 토큰을 생성하는 순간 답변할 내용은 다 생성하
였으니 더 이상 생성을 할 필요가 없으므로 생성을 중단시키는 일종의 신호 역할을 합니다. 라마에서는
"<|eot_id|>"에 해당하며 해당 토큰을 토크나이저를 통해 정수 인코딩하여 그 토큰과 맵핑되는 정수
를 eos_token 변수에 저장합니다.
test_inference 함수는 실제로 모델이 텍스트를 생성하는 부분입니다. max_new_tokens=1024
는 최대 1024 개의 새로운 토큰을 생성하도록 제한합니다. 다시 말해 답변은 해당 길이를 넘어서 생성될
수 없습니다. 이제 테스트 데이터 중에서 10 번부터 14 번 샘플까지 모델을 연속으로 호출하여 실제 레이
블과 모델의 예측값을 비교하며 출력해보도록 하겠습니다.

In [ ]:
for prompt, label in zip(prompt_lst[10:15], label_lst[10:15]):
  print(f" response:\n{test_inference(pipe, prompt)}")
  print(f" label:\n{label}")
  print("-"*50)


파인 튜닝된 모델은 레이블과 비교해서도 거의 유사한 내용을 예측하며 준수한 답변을 생성합니다. 그렇
다면 학습이 전혀 되지 않은 기본 모델의 답변은 어떨지 동일한 데이터를 기본 모델에 입력하여 답변을 생
성해보겠습니다.

###10. 기본 모델 테스트
이번에는 LoRA Adapter 를 merge 하지 않은 기본 모델로 테스트 데이터에 대해서 답변을 출력해보겠습니다.

In [ ]:
base_model_id = "NCSOFT/Llama-VARCO-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

for prompt, label in zip(prompt_lst[10:15], label_lst[10:15]):
  print(f" response:\n{test_inference(pipe, prompt)}")
  print(f" label:\n{label}")
  print("-"*50)


파인 튜닝하지 않은 기본 모델은 파인튜닝 모델과는 달리 전반적으로 답변이 정돈되지 못하고 레이블과
비교하면 오답도 많이 포함된 것으로 보입니다. 이처럼 파인 튜닝을 통해 특정 도메인에서 사용되는 거대
언어 모델의 답변 성능을 높일 수 있습니다. 여러분들도 여러분들만의 도메인에 맞는 학습 데이터를 구축
하여 모델의 성능을 높여보시기 바랍니다.
